<a href="https://colab.research.google.com/github/carmea2025-dev/conversor-colab/blob/main/Conversor_BPA_Com_Botoes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# @title Projeto BAHIA QUE PRODUZ E ALIMENTA - Plano de Ação { display-mode: "form" }

from IPython.display import display, HTML, clear_output
import pandas as pd
import io
import requests
from google.colab import files
import ipywidgets as widgets
from openpyxl import load_workbook

# ============================================================
# TÍTULO VISÍVEL MESMO COM CÓDIGO OCULTO
# ============================================================
display(HTML("""
<h2 style="text-align:center; color:#2c3e50; font-family:'Trebuchet MS', sans-serif;">
📘 Projeto <b>BAHIA QUE PRODUZ E ALIMENTA</b> - Plano de Ação
</h2>
<hr style="border:1px solid #ccc;">
"""))

# ============================================================
# URLs DOS ARQUIVOS NO GITHUB
# ============================================================
url_mapeamento = "https://raw.githubusercontent.com/carmea2025-dev/conversor-colab/main/Mapeamento_Colunas.xlsx"
url_modelo = "https://raw.githubusercontent.com/carmea2025-dev/conversor-colab/main/PLANO%20DE%20AÇÃO%20-%20BPA%20-%20FINAL.xlsx"

# ============================================================
# FUNÇÃO PARA BAIXAR ARQUIVOS DO GITHUB
# ============================================================
def baixar_arquivo(url, nome_arquivo):
    r = requests.get(url)
    if r.status_code == 200:
        with open(nome_arquivo, "wb") as f:
            f.write(r.content)
        print(f"✅ {nome_arquivo} baixado com sucesso.")
    else:
        print(f"❌ Erro ao baixar {nome_arquivo}. Verifique o link do GitHub.")

baixar_arquivo(url_mapeamento, "Mapeamento_Colunas.xlsx")
baixar_arquivo(url_modelo, "PLANO_DE_ACAO_BPA_FINAL.xlsx")

# ============================================================
# FUNÇÃO DE CONVERSÃO
# ============================================================
def converter_arquivo(uploaded_file):
    try:
        # Lê os arquivos de apoio
        mapeamento = pd.read_excel("Mapeamento_Colunas.xlsx", engine='openpyxl')
        modelo_df = pd.read_excel("PLANO_DE_ACAO_BPA_FINAL.xlsx", engine='openpyxl')

        nome_arquivo = list(uploaded_file.keys())[0]
        conteudo = uploaded_file[nome_arquivo]['content']

        # Detecta extensão do arquivo enviado
        if nome_arquivo.endswith(".xls"):
            df_bpa = pd.read_excel(io.BytesIO(conteudo), engine='xlrd')
        elif nome_arquivo.endswith(".xlsx"):
            df_bpa = pd.read_excel(io.BytesIO(conteudo), engine='openpyxl')
        else:
            print("⚠️ Formato de arquivo não suportado. Envie .xls ou .xlsx.")
            return

        # Cria o dicionário de mapeamento
        col_map = dict(zip(mapeamento['Coluna_Origem'], mapeamento['Coluna_Destino']))

        # Aplica o mapeamento às colunas do BPA
        df_convertido = df_bpa.rename(columns=col_map)

        # Carrega o modelo com formatação
        wb = load_workbook("PLANO_DE_ACAO_BPA_FINAL.xlsx")
        ws = wb.active

        # Define as colunas fixas que determinam a estrutura e vínculo das ações
        colunas_fixas = ["Nº", "COMPONENTE", "SUBCOMPONENTE", "Responsabilidade", "Ação"]

        # Cria o dataframe final iniciando a partir do modelo (preserva estrutura)
        df_final = modelo_df.copy()

        # Garante que o DataFrame convertido tenha apenas colunas conhecidas
        colunas_modelo = list(modelo_df.columns)
        df_convertido = df_convertido[[c for c in df_convertido.columns if c in colunas_modelo]]

        # Mescla os dados do modelo (estrutura) com o BPA convertido (dados)
        # Mantém correspondência pela coluna "Ação", que é o vínculo principal
        if "Ação" in df_convertido.columns:
            df_final = pd.merge(
                modelo_df,
                df_convertido,
                on="Ação",
                how="left",
                suffixes=("", "_NOVO")
            )

            # Preenche as colunas do modelo com os dados novos onde houver correspondência
            for col in colunas_modelo:
                col_novo = col + "_NOVO"
                if col_novo in df_final.columns:
                    df_final[col] = df_final[col_novo].combine_first(df_final[col])
                    df_final.drop(columns=[col_novo], inplace=True, errors="ignore")

        # Regrava o conteúdo no modelo Excel mantendo formatação
        for col_idx, col_name in enumerate(colunas_modelo, start=1):
            if col_name not in df_final.columns:
                continue
            valores = df_final[col_name].tolist()
            for row_idx, valor in enumerate(valores, start=2):
                ws.cell(row=row_idx, column=col_idx, value=valor)

        # Salva o arquivo final com a formatação original
        nome_saida = "PLANO DE AÇÃO - BPA - CONVERTIDO.xlsx"
        wb.save(nome_saida)
        print("✅ Conversão concluída com sucesso! Estrutura e formatação preservadas.")
        files.download(nome_saida)

    except Exception as e:
        print(f"❌ Erro durante a conversão: {e}")

# ============================================================
# INTERFACE NO COLAB
# ============================================================
def interface():
    upload_button = widgets.FileUpload(accept=".xls,.xlsx", multiple=False)
    convert_button = widgets.Button(description="Converter e Baixar", button_style="success")

    def ao_clicar(_):
        clear_output(wait=True)
        display(HTML("""
        <h2 style="text-align:center; color:#2c3e50; font-family:'Trebuchet MS', sans-serif;">
        📘 Projeto <b>BAHIA QUE PRODUZ E ALIMENTA</b> - Plano de Ação
        </h2>
        <hr style="border:1px solid #ccc;">
        """))
        display(upload_button, convert_button)
        if upload_button.value:
            converter_arquivo(upload_button.value)
        else:
            print("⚠️ Envie um arquivo BPA primeiro.")

    convert_button.on_click(ao_clicar)
    display(upload_button, convert_button)

# ============================================================
# EXECUTA A INTERFACE
# ============================================================
interface()


FileUpload(value={'PLANO DE AÇÃO - BPA.xlsx': {'metadata': {'name': 'PLANO DE AÇÃO - BPA.xlsx', 'type': 'appli…

Button(button_style='success', description='Converter e Baixar', style=ButtonStyle())

✅ Conversão concluída com sucesso! Estrutura e formatação preservadas.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>